In [1]:
import pandas as pd
import numpy as np

# load data

In [12]:
# data path 
path_ext_data = '/Users/trusch/Box/COVID-19 Adolphs Lab/core_analysis/raw_data/'

# case and dath data
county_cases_deaths = pd.read_csv(path_ext_data + 'refined_us_daily_cases_deaths_by_county.csv')
state_cases_deaths = pd.read_csv(path_ext_data + 'refined_us_daily_cases_deaths_by_state.csv')

# labour statistics
county_labour = pd.read_csv(path_ext_data + 'refined_labor_data_by_county.csv')
#county_labour = pd.read_csv(path_ext_data + 'refined_labor_data_by_state.csv')

# restriction data
county_bar_closure = pd.read_csv(path_ext_data + 'refined_bar_closure_data_county.csv')
county_gather_ban = pd.read_csv(path_ext_data + 'refined_gathering_ban_data_county.csv')
county_general_closure = pd.read_csv(path_ext_data + 'refined_general_closure_data_county.csv')
county_mask_mandate = pd.read_csv(path_ext_data + 'refined_mask_mandate_data_county.csv')
county_restaurant_closure = pd.read_csv(path_ext_data + 'refined_restaurant_closure_data_county.csv')
county_stay_at_home = pd.read_csv(path_ext_data + 'refined_stay_at_home_order_data_county.csv')

county_bar_closure = pd.read_csv(path_ext_data + 'refined_bar_closure_data_state.csv')
county_gather_ban = pd.read_csv(path_ext_data + 'refined_gathering_ban_data_state.csv')
county_general_closure = pd.read_csv(path_ext_data + 'refined_general_closure_data_state.csv')
county_mask_mandate = pd.read_csv(path_ext_data + 'refined_mask_mandate_data_state.csv')
county_restaurant_closure = pd.read_csv(path_ext_data + 'refined_restaurant_closure_data_state.csv')
county_stay_at_home = pd.read_csv(path_ext_data + 'refined_stay_at_home_order_data_state.csv')

# dates for each wave -7 and +1 days, i.e. Sat(pre wave) to Mon (post wave)
wave_dates = pd.read_csv('/Users/trusch/Box/COVID-19 Adolphs Lab/core_analysis/covid_measures/wave_timecourse.csv')
wave_dates = wave_dates.set_index('wave', drop = True)


# wave-by-wave covid numbers

In [16]:
for w in wave_dates.index:
    county_cases_deaths.loc[county_cases_deaths.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w

# select data: 7 days preceding each wave
tmp_df = pd.melt(wave_dates.iloc[:,1:-3])
pre_wave_dates = list(tmp_df['value'])


county_cases_deaths = county_cases_deaths.loc[county_cases_deaths.date.isin(pre_wave_dates),:]
# drop Puerto Rico, Virgin Islands, Northern Mariana Islands, and Guam
county_cases_deaths = county_cases_deaths.loc[~county_cases_deaths.state.isin(['Puerto Rico',
                                                                            'Virgin Islands',
                                                                            'Northern Mariana Islands',
                                                                            'Guam']),:]
# wave-by-wave average
wave_county_cases_deaths = pd.DataFrame(county_cases_deaths.groupby(['county', 'state', 'wave']).mean()).reset_index()
wave_county_cases_deaths

,county,state,wave,daily_cases,seven_day_case_avg,daily_case_avg_per_100k,daily_deaths,seven_day_death_avg,daily_death_avg_per_100k
0,Abbeville,South Carolina,1.0,0.333333,0.488333,1.991667,0.0,0.000000,0.000000
1,Abbeville,South Carolina,2.0,0.166667,0.403333,1.651667,0.0,0.000000,0.000000
2,Abbeville,South Carolina,3.0,0.166667,0.556667,2.266667,0.0,0.000000,0.000000
3,Abbeville,South Carolina,4.0,1.500000,1.523333,6.211667,0.0,0.000000,0.000000
4,Abbeville,South Carolina,5.0,0.500000,0.571667,2.330000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
47793,Ziebach,South Dakota,12.0,0.333333,0.428333,15.548333,0.0,0.000000,0.000000
47794,Ziebach,South Dakota,13.0,0.833333,0.906667,32.826667,0.0,0.140000,5.180000
47795,Ziebach,South Dakota,14.0,2.833333,3.358333,121.811667,0.0,0.000000,0.000000
47796,Ziebach,South Dakota,15.0,10.833333,6.573333,238.440000,0.0,0.046667,1.726667


# month-by-month labour statistics

In [14]:
# drop Puerto Rico
county_labour = county_labour.loc[~county_labour.state.isin(['Puerto Rico']),:]

# labour data is only published on a monthly basis --> we define the month of the Saturday
# preceding a wave as the month preceding a wave
pre_wave_month = list(wave_dates['SAT-7'].str.split('-'))
pre_wave_month  = pd.DataFrame(pre_wave_month, columns = ['year', 'month', 'day'])
pre_wave_month = pre_wave_month.set_index(wave_dates.index, drop = True)

county_labour_wave = pd.DataFrame()
for w in pre_wave_month.index:
    wave_month = pre_wave_month.loc[w,'year'] + '-' + pre_wave_month.loc[w,'month']
    tmp_labour = county_labour.loc[county_labour.date == wave_month, :].copy()
    tmp_labour['wave'] = w
    county_labour_wave= county_labour_wave.append(tmp_labour)
county_labour_wave.reset_index(drop = True, inplace= True)    

# restrictions

In [ ]:
county_bar_closure = county_bar_closure.loc[~county_bar_closure.state.isin(['Puerto Rico']),:]
county_gather_ban = county_gather_ban.loc[~county_gather_ban.state.isin(['Puerto Rico']),:]
county_general_closure = county_general_closure.loc[~county_general_closure.state.isin(['Puerto Rico']),:]
county_mask_mandate = county_mask_mandate.loc[~county_mask_mandate.state.isin(['Puerto Rico']),:]
county_restaurant_closure = county_restaurant_closure.loc[~county_restaurant_closure.state.isin(['Puerto Rico']),:]
county_stay_at_home = county_stay_at_home.loc[~county_stay_at_home.state.isin(['Puerto Rico']),:]

for w in wave_dates.index:
    county_bar_closure.loc[county_bar_closure.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w
    county_gather_ban.loc[county_gather_ban.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w
    county_general_closure.loc[county_general_closure.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w
    county_mask_mandate.loc[county_mask_mandate.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w
    county_restaurant_closure.loc[county_restaurant_closure.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w
    county_stay_at_home.loc[county_stay_at_home.date.isin(list(wave_dates.loc[w,:])), 'wave'] = w
     
county_bar_closure = county_bar_closure.loc[county_bar_closure.date.isin(wave_dates['MON-5'].values),:]
county_gather_ban = county_gather_ban.loc[county_gather_ban.date.isin(wave_dates['MON-5'].values),:]
county_general_closure = county_general_closure.loc[county_general_closure.date.isin(wave_dates['MON-5'].values),:]
county_mask_mandate = county_mask_mandate.loc[county_mask_mandate.date.isin(wave_dates['MON-5'].values),:]
county_restaurant_closure = county_restaurant_closure.loc[county_restaurant_closure.date.isin(wave_dates['MON-5'].values),:]
county_stay_at_home = county_stay_at_home.loc[county_stay_at_home.date.isin(wave_dates['FRI-1'].values),:]


In [ ]:
wave_dates